In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from tensorflow.keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split
import os
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from PIL import Image


MODEL DESIGN

In [ ]:
model = models.Sequential()
model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=(300, 300, 1)))
model.add(layers.MaxPooling2D((4, 4)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))

model.add(layers.Flatten())

model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dropout(0.2))
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(16, activation='relu'))

model.add(layers.Dense(2, activation='softmax'))


model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

#model summary
model.summary()

Model: "sequential_21"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_60 (Conv2D)          (None, 298, 298, 64)      640       
                                                                 
 max_pooling2d_55 (MaxPooli  (None, 74, 74, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_61 (Conv2D)          (None, 72, 72, 64)        36928     
                                                                 
 max_pooling2d_56 (MaxPooli  (None, 36, 36, 64)        0         
 ng2D)                                                           
                                                                 
 conv2d_62 (Conv2D)          (None, 34, 34, 32)        18464     
                                                                 
 max_pooling2d_57 (MaxPooli  (None, 17, 17, 32)      

In [ ]:
def resize_image(image_path):
    img = load_img(image_path, target_size=(300, 300), color_mode='grayscale')
    img_array = img_to_array(img)
    img_array /= 255.0  # Normalize pixel values to between 0 and 1
    return img_array


def resizeimg(folderpath: str,imgname: str):
  img = Image.open(folderpath+imgname)
  img.thumbnail((300,300))
  img.save(folderpath[:-1]+'_/'+imgname)
  m,n = img.size
  target_size = (300, 300)
  if (m!=300)or(n!=300):
    if m>n:
      target_size = (m,m)
    else:
      target_size = (n,n)
  else:
    return;
  image = cv2.imread(folderpath[:-1]+'_/'+imgname)
  vertical_padding = (target_size[0] - image.shape[0]) // 2
  horizontal_padding = (target_size[1] - image.shape[1]) // 2

  padded_image = cv2.copyMakeBorder(image, vertical_padding, vertical_padding, horizontal_padding, horizontal_padding, cv2.BORDER_CONSTANT)
  cv2.imwrite(folderpath[:-1]+'_/'+imgname,padded_image)


In [ ]:
metadatapath = "updated.csv"
metadata = pd.read_csv(metadatapath)
cancerpath = "Brain Tumor_/"
normalpath = "Healthy_/"
count=0
paths=[]
labels=[]

metadata.drop(metadata.columns[[0]], axis=1, inplace=True)

for i in range(0,4519):
  if (metadata.iloc[i]['class']=='cancer' ):
    paths.append(cancerpath+metadata.iloc[i]['Filename'])
    labels.append('cancer')
  else:
    paths.append(normalpath+metadata.iloc[i]['Filename'])
    labels.append('normal')
df = pd.DataFrame({
    'path':paths,
    'label':labels
})
print(df)

TEST TRAIN VAL SPLIT

In [ ]:
train_df, test_df = train_test_split(df, test_size=0.3, random_state=42)
train_df, val_df = train_test_split(train_df, test_size=0.1, random_state=42)

train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_dataframe(
    train_df,
    x_col='path',
    y_col='label',
    target_size=(300, 300),
    batch_size=4,
    class_mode='binary',  # or 'binary' for binary classification
    color_mode='grayscale',  # or 'rgb' for RGB images
    shuffle=True
)

test_generator = test_datagen.flow_from_dataframe(
    test_df,
    x_col='path',
    y_col='label',
    target_size=(300, 300),
    batch_size=1,
    class_mode='binary',
    color_mode='grayscale',
    shuffle=False
)
val_generator = val_datagen.flow_from_dataframe(
    val_df,
    x_col='path',
    y_col='label',
    target_size=(300, 300),
    batch_size=1,
    class_mode='binary',
    color_mode='grayscale',
    shuffle=False
)



Found 2846 validated image filenames belonging to 2 classes.
Found 1356 validated image filenames belonging to 2 classes.
Found 317 validated image filenames belonging to 2 classes.


TRAINING THE MODEL

In [ ]:
#checkpoint = ModelCheckpoint('/content/drive/MyDrive/lund/model10.h5', save_best_only=True)

model.fit(
    train_generator,
    epochs=1,
    validation_data=val_generator
    #callbacks=[checkpoint]
)

# Visualize training and validation loss
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Training and Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.show()

# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_generator)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

In [ ]:
test_loss, test_accuracy = model.evaluate(val_generator)
print(f'Test Loss: {test_loss}, Test Accuracy: {test_accuracy}')

317/317 [==============================] - 15s 47ms/step - loss: 0.1176 - accuracy: 0.9653
Test Loss: 0.11757364869117737, Test Accuracy: 0.965299665927887


F1 SCORE CALCULATION

In [ ]:
predictions = model.predict(test_generator)
labels = test_generator.labels

1356/1356 [==============================] - 277s 205ms/step


In [ ]:
print(np.argmax(predictions,axis =1))
print(np.array(labels))

[1 0 1 ... 0 0 0]
[1 0 1 ... 0 0 0]


In [ ]:
from sklearn.metrics import precision_score, recall_score

precision = precision_score(labels, np.argmax(predictions,axis =1) , average='weighted')
recall = recall_score(labels, np.argmax(predictions,axis =1), average='weighted')

print(f'Precision: {precision}')
print(f'Recall: {recall}')

Precision: 0.9411158731365409
Recall: 0.9402654867256637


In [ ]:
print("F1 score : ")
print(2*precision*recall/(precision+recall));

F1 score : 
0.9406904877433132


In [ ]:
model.save("model.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


USER TESTING

In [ ]:
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.preprocessing.image import img_to_array, load_img
from PIL import Image
import cv2
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import numpy as np

def displayimg(path):
  img = mpimg.imread(path)
  plt.imshow(img)
  plt.axis('off')
  plt.show()
def process_after_save(folderpath: str,imgname: str):
  img = Image.open(folderpath+imgname)
  img.thumbnail((300,300))
  img.save(folderpath+imgname)
  m,n = img.size
  target_size = (300, 300)
  if (m!=300)or(n!=300):
    if m>n:
      target_size = (m,m)
    else:
      target_size = (n,n)
  else:
    return;
  image = cv2.imread(folderpath+imgname)
  vertical_padding = (target_size[0] - image.shape[0]) // 2
  horizontal_padding = (target_size[1] - image.shape[1]) // 2

  padded_image = cv2.copyMakeBorder(image, vertical_padding, vertical_padding, horizontal_padding, horizontal_padding, cv2.BORDER_CONSTANT)
  cv2.imwrite(folderpath+imgname,padded_image)


def process_before_model(image_path):
    img = load_img(image_path, target_size=(300, 300), color_mode='grayscale')
    img_array = img_to_array(img)
    img_array /= 255.0  # Normalize pixel values to between 0 and 1
    return img_array


path = "s.png"
displayimg(path)
img=load_img(path)
arry = img_to_array(img)
print(arry.shape)

process_after_save("","Not Cancer  (1008).jpg")
displayimg(path)
img=load_img(path)
print(img_to_array(img).shape)

finarr = process_before_model(path)

model2 = load_model("modfinaltt.h5")

print(finarr.shape)
finarr = np.expand_dims(finarr, axis=0)
val = model2.predict(finarr)
print(val)
for x in np.argmax(val,axis=1):
  if(x==0):
    print("tumor")
  else:
    print("no tumor")

